<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NB-description" data-toc-modified-id="NB-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NB description</a></span></li><li><span><a href="#The-dotsPositions.csv-data" data-toc-modified-id="The-dotsPositions.csv-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The dotsPositions.csv data</a></span></li><li><span><a href="#Write-a-dotsDB-HDF5-file" data-toc-modified-id="Write-a-dotsDB-HDF5-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Write a dotsDB HDF5 file</a></span><ul class="toc-item"><li><span><a href="#Write-HDF5-file" data-toc-modified-id="Write-HDF5-file-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Write HDF5 file</a></span></li></ul></li></ul></div>

# NB description
date: 04 Dec 2019  
This notebook contains code that:
- builds an HDF5 dotsDB database off of dotsPositions.csv files from the Fall 2019 data (subjects 10-13)

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
import seaborn as sns
import h5py     
import os.path

# add location of custom modules to path
sys.path.insert(0,'../modules/')
sys.path.insert(0,'../modules/dots_db/dotsDB/')

# custom modules
import dotsDB as ddb
import motionenergy as kiani_me
import stimulus as stim
import ME_functions as my_me

# The dotsPositions.csv data
With the current pipeline, on the day of the session, a `.csv` file is written to disk with the FIRA data. Then, in the `motion_energy_Adrian` repo, I have MATLAB functions `reproduce_dots` and `batch_reproduce_dots` that write `_dotsPositions.csv` files to disk (one file per session).

The first step here is to loop through the completed sessions, perform a `join` of the dots and fira data, and update a global `.csv` file, as well as session-specific `.csv` files, called `labeled_dots_<timestamp>.csv`.

In [ ]:
!find /home/adrian/SingleCP_DotsReversal/Fall2019/raw -name "*dotsPositions.csv" -print

In [ ]:
!find /home/adrian/SingleCP_DotsReversal/Fall2019/raw -name "completed*100*.csv" -print

In [2]:
def label_dots(timestamps, global_labeled_dots_filename, data_folder):
    """
    fetches dots data outputted by MATLAB (the _dotsPositions.csv files) for specified session timestamps, adds
    relevant fira data (join operation) and appends resulting 'labeled_dots' dataframe to the 
    global_labeled_dots_filename.
    :param timestamps: list or tuple of strings of the form '2019_11_05_16_19'
    :param global_labeled_dots_filename: string with full path and filename for global .csv file to write to
    :param data_folder: string with path to folder '.../raw/' where fira and dotsPositions .csv data reside.
    :return: None, but writes to file
    """
    list_of_labeled_dots_dataframes = []
    for ts in timestamps:
        folder = data_folder + ts + '/'
        fira = pd.read_csv(folder + 'completed4AFCtrials_task100_date_' + ts + '.csv')
        dots = pd.read_csv(folder + ts + '_dotsPositions.csv')
        dots = dots[dots['isActive'] == 1]
        del dots['isActive'], dots['taskID'], dots['isCoherent']
        try:
            assert fira.index.min() == 0 and fira.index.max() == 819 and len(fira.index) == 820
            assert dots['trialIx'].min() == 0 and dots['trialIx'].max() == 819
        except AssertionError:
            print(f'assert failed with timestamp {ts}')
            continue
        labeled_dots = dots.join(fira, on="trialIx")
        labeled_dots['trueVD'] = labeled_dots['dotsOff'] - labeled_dots['dotsOn']
        labeled_dots['presenceCP'] = labeled_dots['reversal'] > 0
        to_drop = ['trialIndex', 'RT', 'cpRT', 'dirCorrect', 'cpCorrect', 
            'randSeedBase', 'fixationOn', 'fixationStart', 'targetOn',
            'choiceTime', 'cpChoiceTime', 'blankScreen', 'feedbackOn', 
            'cpScreenOn', 'dummyBlank', 'finalDuration', 'dotsOn', 'dotsOff']
        labeled_dots.drop(columns=to_drop, inplace=True)
        to_rename = {
            'duration': 'viewingDuration',
            'direction': 'initDirection',
        }
        labeled_dots.rename(columns=to_rename, inplace=True)
        labeled_dots.dropna(subset=['dirChoice'], inplace=True)
        list_of_labeled_dots_dataframes.append(labeled_dots)
        
    # only write to file if list of dataframes is not empty
    if list_of_labeled_dots_dataframes:
        full_labeled_dots = pd.concat(list_of_labeled_dots_dataframes)
        if os.path.exists(global_labeled_dots_filename):
            full_labeled_dots.to_csv(global_labeled_dots_filename, index=False, mode='a+', header=False)
        else:
            full_labeled_dots.to_csv(global_labeled_dots_filename, index=False, mode='a+', header=True)
            
    return None

In [ ]:
'''
list of written data timestamps:
'2019_11_06_12_43'
'2019_11_05_16_19',
'2019_11_20_15_34',
'2019_11_19_13_15',
'2019_11_05_13_18',
'2019_11_26_13_11',
'2019_11_25_16_12',

list of problematic timestamps that were NOT written
'2019_11_05_10_27',
'''
TIMESTAMPS = ()

In [3]:
DATA_FOLDER = '/home/adrian/SingleCP_DotsReversal/Fall2019/raw/'

The fira dataframe has index ranging from 0 to 819. The next step is to create a "foreign key" to this index into the dots dataframe.

In [4]:
DOTS_LABELED = '/home/adrian/SingleCP_DotsReversal/Fall2019/processed/dots_fall_2019_v1.csv'

In [ ]:
label_dots(TIMESTAMPS, DOTS_LABELED, DATA_FOLDER)

In [5]:
def inspect_csv(df):
    """df is a pandas.DataFrame"""
    print(df.head())
    print(len(df))
    print(np.unique(df['taskID']))
    try:
        print(np.unique(df['pilotID']))
    except KeyError:
        print(np.unique(df['subject']))

# Write a dotsDB HDF5 file
Now that all the dotsPositions.csv data is collected into a single global .csv file, I wish to dump it all into an hdf5 database.

Several actions need to be implemented.
1. For each trial in the dotsPositions.csv data, I need to know: _coherence_, _viewing duration_, _presenceCP_, _direction_, _subject_, _block_ (_probCP_). For this, I will assume that the `trialEnd` (from FIRA) and `seqDumpTime` (from dotsPositions) timestamps are in the same unit.
2. I need to decide how to organize my dotsDB hierarchically. Example is `subj15/probCP0.1/coh0/ansleft/CPno/VD100`

In [6]:
def get_trial_params(df):
    """coherence, viewing duration, presenceCP, direction, subject, block, probCP"""
    coh = df['coherence'].values[0]
    vd = df['viewingDuration'].values[0]
    pcp = df['presenceCP'].values[0]
    idir = df['initDirection'].values[0]
    subj = df['subject'].values[0]
    block = df['block'].values[0]
    Pcp = df['probCP'].values[0]
    return coh, vd, pcp, idir, subj, block, Pcp

In [7]:
def get_trial_from_dots_ts(dot_ts, trials_ts, trials_df):
    trial_dump_time = np.min(trials_ts[trials_ts>dot_ts])
    assert trial_dump_time - dot_ts < .5, 'trialEnd occurs more than 0.5 sec after seqDumpTime'
    return trials_df[trials_df['trialEnd'] == trial_dump_time]

In [8]:
def add_trial_params(row, t, trials):
    """
    function that adds appropriate values to trial parameter columns in dots dataframe
    :param row: row from dataframe
    :param t: dataframe with FIRA data
    :param trials: numpy array of trialEnd timestamps (scalars)
    """
    time = row['seqDumpTime']
    try:
        trial = get_trial_from_dots_ts(time, trials, t)
    except AssertionError:
        print(f'0.5 sec margin failed at row {row.name}')
        return row
    c,v,p,i,s,b,P = get_trial_params(trial)
    row['coherence'] = c
    row['viewingDuration'] = v
    row['presenceCP'] = p
    row['initDirection'] = i
    row['subject'] = s
    row['block'] = b
    row['probCP'] = P
    return row

def set_nans(df):
    if 'isActive' in df:
        del df['isActive']
    df['coherence'] = np.nan
    df['viewingDuration'] = np.nan
    df['presenceCP'] = np.nan
    df['initDirection'] = np.nan
    df['subject'] = np.nan
    df['block'] = np.nan
    df['probCP'] = np.nan
    return df

So far so good, for a given `seqDumpTime` value, I am able to recover the trial's parameters. All that remains to do is to add columns to the dots dataframe (and remove the `isActive` one).

**Following cell is SLOW! Around 30 min**

I forgot to record true duration and subject's choice!
Rebelote...

I should have added `dirChoice` instead of `cpChoice`!!!

Now, it turns out some rows have a `nan` value in the `dirChoice` column!
I will drop them!

## Write HDF5 file
Now, I need to write an HDF5 file with the structure:
`subj15/probCP0.1/coh0/ansleft/CPno/VD100`.

I need to:
- loop through the trials contained in the dots DF
- port the dots data to dotsDB format
- write to file

In [9]:
dots = pd.read_csv(DOTS_LABELED)

In [10]:
dots.head()

,xpos,ypos,frameIdx,trialIx,taskID,trialStart,trialEnd,dirChoice,cpChoice,initDirection,coherence,finalCPTime,subject,date,probCP,reversal,viewingDuration,trueVD,presenceCP
0,0.915860,0.251297,1,0,100,2.215517e+06,2.215519e+06,0,0,180,18,NaN,11,201911061243,0.7,0.0,0.4,0.424559,False
1,0.283857,0.940118,1,0,100,2.215517e+06,2.215519e+06,0,0,180,18,NaN,11,201911061243,0.7,0.0,0.4,0.424559,False
2,0.961799,0.232799,1,0,100,2.215517e+06,2.215519e+06,0,0,180,18,NaN,11,201911061243,0.7,0.0,0.4,0.424559,False
3,0.961470,0.362006,1,0,100,2.215517e+06,2.215519e+06,0,0,180,18,NaN,11,201911061243,0.7,0.0,0.4,0.424559,False
4,0.559943,0.692933,1,0,100,2.215517e+06,2.215519e+06,0,0,180,18,NaN,11,201911061243,0.7,0.0,0.4,0.424559,False


In [11]:
dots.shape

(7138854, 19)

In [12]:
dots.columns

Index(['xpos', 'ypos', 'frameIdx', 'trialIx', 'taskID', 'trialStart',
       'trialEnd', 'dirChoice', 'cpChoice', 'initDirection', 'coherence',
       'finalCPTime', 'subject', 'date', 'probCP', 'reversal',
       'viewingDuration', 'trueVD', 'presenceCP'],
      dtype='object')

In [13]:
gb = dots.groupby('trialEnd')  # recall gb.get_group() and gb['frameIdx'].max()

At this stage, I would like to know the max value of `frameIdx` in each trial.

In [14]:
# Needs to be re-written
def get_frames(df):
    """
    get the dots data as a list of numpy arrays, as dotsDB requires them
    """
    # (could/should probably be re-written with groupby and apply...)
    num_frames = np.max(df["frameIdx"]).astype(int)
    assert not np.isnan(num_frames), 'NaN num_frames'
    list_of_frames = []
    for fr in range(num_frames):
        frame_data = df[df["frameIdx"] == (fr+1)]
        list_of_frames.append(np.array(frame_data[['ypos','xpos']]))  # here I swap xpos with ypos for dotsDB
    return list_of_frames

def get_group_name(df):
    """
    get the trial's parameters, and therefore the HDF5 group where the data should be appended
    """       
    # get HDF5 group name
    
    def choice(c):
        """
        :param c: (int) either 0 for 'left' or 1 for 'right'
        :return: (str) either '/ansright' or '/ansleft'
        """
        if c == 1:
            return '/ansright' 
        elif c == 0:
            return '/ansleft'
        else:
            raise ValueError(f'unexpected choice value {c}')
            
    def chgepoint(c):
        """
        :param c: (bool) whether the trial contains a CP or not
        :return: (str) either '/CPyes' or '/CPno'
        """
        return '/CPyes' if c else '/CPno'
    
    def viewdur(v):
        """
        :param v: (float or int) viewing duration in seconds
        :return: (str) e.g. '/VD200'
        """
        return '/VD' + str(int(1000*v))
    
    def direction(d):
        """
        :param d: (int) either 180 for left or 0 for rightward moving dots (at start of trial)
        :return: (str) either 'left' or 'right'
        """
        return 'left' if d else 'right'
    
    ss, pp, cc, ch, cp, vd, di, cpt= df[['subject', 'probCP', 'coherence', 'dirChoice', 'presenceCP', 
                               'viewingDuration', 'initDirection', 'finalCPTime']].values[0,:]
    
    ss = str(ss)
    group_name = '/subj' + ss + \
                 '/probCP' + str(pp) + \
                 '/coh' + str(cc) + \
                 choice(ch) + \
                 chgepoint(cp) + \
                 viewdur(vd) + '/' + \
                 direction(di)
                 
    vals = {'coh': cc, 
            'subject': ss, 
            'probCP': pp, 
            'dirChoice': ch,
            'presenceCP': cp,
            'viewingDuration': vd, 
            'initDirection': direction(di),
            'cpTime': cpt}
    
    return group_name, vals

def write_dots_to_file(df, hdf5_file):
    """
    The aim of this function is to write the dots info contained in the pandas.DataFrame df to a dotsDB HDF5 file.
    df should only contain data about a single trial. 
    
    head on df looks like this
    xpos	ypos	isCoherent	frameIdx	seqDumpTime	pilotID	taskID	coherence	viewingDuration	presenceCP	initDirection	subject	block	probCP	cpChoice	trueVD	trialEnd	dirChoice
	0.722093	0.416122	1.0	1.0	1069.27719	2.0	3.0	48.5	0.3	0.0	180.0	S1	Block2	0.0	NaN	0.318517	1069.535562	0.0
	0.681785	0.356234	1.0	1.0	1069.27719	2.0	3.0	48.5	0.3	0.0	180.0	S1	Block2	0.0	NaN	0.318517	1069.535562	0.0
	0.445828	0.914470	1.0	1.0	1069.27719	2.0	3.0	48.5	0.3	0.0	180.0	S1	Block2	0.0	NaN	0.318517	1069.535562	0.0
	0.833181	0.112126	1.0	1.0	1069.27719	2.0	3.0	48.5	0.3	0.0	180.0	S1	Block2	0.0	NaN	0.318517	1069.535562	0.0
	0.013516	0.354543	1.0	1.0	1069.27719	2.0	3.0	48.5	0.3	0.0	180.0	S1	Block2	0.0	NaN	0.318517	1069.535562	0.0
    """
    frames = get_frames(df)
    gn, params = get_group_name(df)
    
    # exit function if number of frames too different from theoretical one 
    vd = params['viewingDuration']
    num_frames = len(frames)
    if abs(num_frames-vd*60) > 5:
        tr = df['trialEnd'].values[0]
        print(f'trial with trialEnd {tr} not written; discrepancy num_frames {num_frames} and VD {vd}')
        return None
    
    cptime = params['cpTime'] if params['presenceCP'] else None
    parameters = dict(speed=5, 
                      density=90, 
                      coh_mean=params['coh'], 
                      coh_stdev=10, 
                      direction=params['initDirection'],
                      num_frames=np.max(df["frameIdx"]).astype(int),
                      diameter=5, 
                      pixels_per_degree=(55.4612 / 2), 
                      dot_size_in_pxs=3, 
                      cp_time=cptime)
    
    stimulus = ddb.DotsStimulus(**parameters)
    
    try:
        ddb.write_stimulus_to_file(stimulus, 1, hdf5_file, 
                                   pre_generated_stimulus=[frames],
                                   group_name=gn, append_to_group=True, max_trials=50)
    except TypeError:
        print(f'group name {gn}')
        print(f'type(frames) = {type(frames)}, len(frames)= {len(frames)}, frames[0].shape = {frames[0].shape}')
        raise

In [ ]:
# # get the first two seqDumpTime values for toy example
# counter = 0
# for ix in gb.groups.keys():
#     counter += 1
#     if counter == 6:
#         break
#     write_dots_to_file(dots[dots['seqDumpTime']==ix], 'test_pilot.h5')

Following cell takes a bit under 9 min

In [15]:
# Recall func is called twice the first time!
if False:  # just a failsafe not to run this cell by mistake
    _ = gb.apply(write_dots_to_file, 'fall2019_v1.h5')

# no need to go in manually and delete the first entry in the dataset corresponding to 
# the first group element gb.groups.keys()[0]

doubling length of dset /subj10/probCP0.7/coh61/ansright/CPyes/VD400/right/px in get_ix()


In [17]:
tend = list(gb.groups.keys())[0]
fira = pd.read_csv(DATA_FOLDER + 'full_2019_12_02.csv')
fira.head()

,taskID,trialIndex,trialStart,trialEnd,RT,cpRT,dirChoice,cpChoice,dirCorrect,cpCorrect,...,blankScreen,feedbackOn,subject,date,probCP,cpScreenOn,dummyBlank,reversal,duration,finalDuration
0,100,1,2.131056e+06,2.131063e+06,0.213185,2.864000,0,0,1,1,...,2.131063e+06,NaN,10,201911051318,0.7,3.290608,6.155014,0.0,0.40,NaN
1,100,2,2.131064e+06,2.131066e+06,0.458183,0.815998,1,0,0,0,...,2.131066e+06,NaN,10,201911051318,0.7,1.761605,2.575164,0.2,0.25,NaN
2,100,3,2.131067e+06,2.131070e+06,0.413732,0.735999,1,0,1,1,...,2.131070e+06,NaN,10,201911051318,0.7,1.412050,2.140863,0.0,0.20,NaN
3,100,4,2.131071e+06,2.131074e+06,0.357103,1.112000,1,0,0,1,...,2.131074e+06,NaN,10,201911051318,0.7,1.975471,3.077166,0.0,0.40,NaN
4,100,5,2.131075e+06,2.131077e+06,0.494251,0.567998,1,0,0,0,...,2.131077e+06,NaN,10,201911051318,0.7,1.485211,2.044533,0.2,0.25,NaN


In [18]:
fira[fira['trialEnd'] == tend]

,taskID,trialIndex,trialStart,trialEnd,RT,cpRT,dirChoice,cpChoice,dirCorrect,cpCorrect,...,blankScreen,feedbackOn,subject,date,probCP,cpScreenOn,dummyBlank,reversal,duration,finalDuration
3255,100,796,525427.348302,525437.700952,1.930979,5.080008,1,0,1,1,...,525437.696367,NaN,12,201911191315,0.3,5.046648,10.129862,0.0,0.3,NaN


In [22]:
fira.dtypes

taskID             int64
trialIndex         int64
trialStart       float64
trialEnd         float64
RT               float64
cpRT             float64
dirChoice          int64
cpChoice           int64
dirCorrect         int64
cpCorrect          int64
direction          int64
coherence          int64
randSeedBase       int64
fixationOn       float64
fixationStart    float64
targetOn         float64
dotsOn           float64
finalCPTime      float64
dotsOff          float64
choiceTime       float64
cpChoiceTime     float64
blankScreen      float64
feedbackOn       float64
subject            int64
date               int64
probCP           float64
cpScreenOn       float64
dummyBlank       float64
reversal         float64
duration         float64
finalDuration    float64
dtype: object

In [23]:
fira[
    (fira['subject'] == 10) & (fira['probCP'] == 0.7) & (fira['coherence'] == 61) \
    & (fira['dirChoice'] == 0) & (fira['reversal'] == 0.2) & (fira['duration'] == .4) & \
    (fira['direction'] == 0)
]

,taskID,trialIndex,trialStart,trialEnd,RT,cpRT,dirChoice,cpChoice,dirCorrect,cpCorrect,...,blankScreen,feedbackOn,subject,date,probCP,cpScreenOn,dummyBlank,reversal,duration,finalDuration
11,100,12,2.131138e+06,2.131140e+06,0.520079,0.672000,0,1,0,1,...,2.131140e+06,NaN,10,201911051318,0.7,1.683229,2.361195,0.2,0.4,NaN
53,100,54,2.131313e+06,2.131315e+06,0.803478,0.568011,0,1,0,1,...,2.131315e+06,NaN,10,201911051318,0.7,2.104251,2.680522,0.2,0.4,NaN
145,100,146,2.131827e+06,2.131832e+06,1.926990,0.672005,0,1,0,1,...,2.131832e+06,NaN,10,201911051318,0.7,4.908984,5.586950,0.2,0.4,NaN
530,100,531,2.134998e+06,2.135001e+06,0.814381,0.776001,0,1,0,1,...,2.135001e+06,NaN,10,201911051318,0.7,2.097207,2.859919,0.2,0.4,NaN


The fact that only 4 trials correspond to the dataset for which the HDF5 length was doubled during the dump procedure is worrying.